In [12]:
from imp4nb import *

cnx = lite.connect('data\\quandan.db')

start = time.clock()
lastdate = pd.to_datetime(pd.read_sql("select max(日期) from xiaoshoumingxi",cnx).iloc[0][0])
print(type(lastdate))
print(lastdate)
df = pd.read_sql_query("select 日期,strftime('%Y%m',日期) as 年月,customer.往来单位编号 as 客户编码,sum(金额) as 金额,substr(customer.往来单位编号,1,2) as 区域 ," \
         "substr(customer.往来单位编号,12,1) as 类型 from xiaoshoumingxi, customer where (customer.往来单位 = xiaoshoumingxi.单位全名) " \
         " group by 日期,单位全名",cnx)
finish1 = time.clock()
descdb(df)
finish2 = time.clock()
print(df[df.日期 >= '2017-12-01'].groupby('客户编码')['金额'].sum())
cnm = lite.connect(':memory:')
df.to_sql('griqidanwei',cnm, if_exists='replace')
dfm = pd.read_sql_query("select sum(金额) as 销售额 from griqidanwei where 日期 >= '2017-12-01' group by 客户编码" ,cnm)
cnm.close()
print(len(dfm))
print(dfm)

print("起始：%f，读取数据构建DataFrame：%f，描述DataFrame：%f。" %(start,finish1-start,finish2-finish1))

# dfcm = df.duplicated()
# print(list(dfcm[dfcm == True].index))
# df =df.drop_duplicates()
# descdb(df)

cnx.close()

<class 'pandas._libs.tslib.Timestamp'>
2017-12-16 00:00:00


163821
                         日期      年月             客户编码     金额  区域 类型
163816  2017-12-16 00:00:00  201712  12300100000SXXD  201.0  12  S
163817  2017-12-16 00:00:00  201712  12400060000BXXA  128.0  12  B
163818  2017-12-16 00:00:00  201712  01200570200SXXD  669.0  01  S
163819  2017-12-16 00:00:00  201712  34100490070SXXD  139.0  34  S
163820  2017-12-16 00:00:00  201712  26300320000SXXD  462.0  26  S
日期       object
年月       object
客户编码     object
金额      float64
区域       object
类型       object
dtype: object
Index(['日期', '年月', '客户编码', '金额', '区域', '类型'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163821 entries, 0 to 163820
Data columns (total 6 columns):
日期      163821 non-null object
年月      163821 non-null object
客户编码    163821 non-null object
金额      163821 non-null float64
区域      163821 non-null object
类型      163821 non-null object
dtypes: float64(1), object(5)
memory usage: 7.5+ MB
None
(163821, 6)
                  金额
count  163821.000000
mean      3

912
          销售额
0      374.20
1      160.00
2      176.00
3      311.70
4      189.00
5      199.00
6      221.00
7      631.00
8      718.90
9      133.00
10      77.00
11     231.00
12     220.00
13     172.00
14     210.00
15     135.30
16     107.00
17     615.00
18     268.00
19     253.00
20     395.00
21     132.00
22     350.80
23    1312.00
24     163.00
25     300.00
26     189.00
27     483.92
28     291.00
29     111.00
..        ...
882     83.20
883    373.00
884    480.00
885    176.00
886    390.00
887    714.50
888    126.00
889    149.00
890    105.00
891   1065.00
892    145.00
893    216.00
894    347.00
895     66.00
896    179.00
897    352.00
898     45.00
899    118.35
900    258.00
901    155.50
902    227.00
903    117.00
904    212.00
905    341.60
906  23008.00
907    742.00
908    145.10
909    -55.80
910   1000.00
911    718.00

[912 rows x 1 columns]
起始：1252.239997，读取数据构建DataFrame：3.332850，描述DataFrame：0.058304。


In [6]:
# 关联客户信息和产品信息
from imp4nb import *

cnx = lite.connect('data\\quandan.db')
# desclitedb(cnx)

# cur = cnx.cursor()
# result = cur.execute('select * from xiaoshoumingxi')
# colnames = [tuple1[0] for tuple1 in result.description]
# print(colnames)
# print(len(result.fetchall()))
# print(result.fetchone()[0])

# df = pd.read_sql_query(
#     "select 日期,职员名称 as 业务主管,单据编号 as 销售单号,商品备注 as 订单单号,单据类型,摘要,xiaoshoumingxi.单位全名 as 客户,customer.往来单位编号 as 编码,"
#     "xiaoshoumingxi.商品全名 as 商品全名,数量,xiaoshoumingxi.单价,金额,substr(customer.往来单位编号,1,2) as 大区 ,"
#     "substr(customer.往来单位编号,1,3) as 小区 ,quyu.分部 as 分部,"
#     "substr(customer.往来单位编号,12,1) as 类型编码,leixing.类型 as 客户类型, product.品牌名称  as 品牌 ,product.品牌分类  as 品牌分类 from xiaoshoumingxi,"
#     "customer,product,quyu,leixing where (customer.往来单位 = xiaoshoumingxi.单位全名)  and (大区 = quyu.区域) "
#     "and (product.商品全名 = xiaoshoumingxi.商品全名) and (类型编码 = leixing.编码) order by 日期 desc", cnx,parse_dates=['日期'])
tm0 = time.clock()

df = pd.read_sql_query("select * from xiaoshoumingxi",cnx)
# descdb(df)
log.info('数据表“销售明细”中有%d条记录' %(len(df)))
tm1 = time.clock()

df = pd.read_sql_query(
    "select 日期,职员名称 as 业务主管,单据编号 as 销售单号,商品备注 as 订单单号,单据类型,摘要,xiaoshoumingxi.单位全名 as 客户,customer.往来单位编号 as 编码,"
    "xiaoshoumingxi.商品全名 as 商品全名,数量,xiaoshoumingxi.单价,金额,substr(customer.往来单位编号,1,2) as 大区 ,"
    "substr(customer.往来单位编号,1,3) as 小区 ,quyu.分部 as 分部,"
    "substr(customer.往来单位编号,12,1) as 类型编码, product.品牌名称  as 品牌 ,product.品牌分类  as 品牌分类 from xiaoshoumingxi,"
    "customer,product,quyu where (customer.往来单位 = xiaoshoumingxi.单位全名)  and (大区 = quyu.区域) "
    "and (product.商品全名 = xiaoshoumingxi.商品全名) order by 日期 desc", cnx)
# descdb(df)
log.info('数据表“销售明细”和“客户”、“品牌”、“区域”关联后有效记录数为：%d条' %(len(df)))
tm2 = time.clock()

df['日期'] = df['日期'].apply(lambda x: pd.to_datetime(x))
log.debug('处理关联客户品牌区域等信息的数据表，把‘日期’处理成了日期格式')
tm3 = time.clock()

# cnm = lite.connect(":memory:")
# df.to_sql(name='alldata', con=cnx, if_exists='replace',chunksize=50000)
df.to_sql(name='alldata', con=cnx, if_exists='replace')
log.info('把关联并规整后的数据写入了“alldata”数据表')
# cnm.close()
tm4 = time.clock()

df = pd.read_sql_query('select 日期,sum(金额) as 销售额 from alldata group by 日期',cnx)
# descdb(df)
tm5 = time.clock()

cnx.close()

print("起始时间：%f，从销售明细读取所有记录获取记录数：%f；"
      "关联客户、品牌和区域信息获得处理后的记录：%f；"
      "处理日期列：%f；"
      "写入内存数据库：%f；"
      "从存储数据库读取汇总销售数据：%f。" %(tm0,tm1-tm0,tm2-tm1,tm3-tm2,tm4-tm3,tm5-tm4))


2017-12-25 01:09:51,180	ewer        : INFO     数据表“销售明细”中有1444120条记录


2017-12-25 01:11:28,556	ewer        : INFO     数据表“销售明细”和“客户”、“品牌”、“区域”关联后有效记录数为：1444120条


2017-12-25 01:12:58,179	ewer        : DEBUG    处理关联客户品牌区域等信息的数据表，把‘日期’处理成了日期格式


2017-12-25 01:22:45,570	ewer        : INFO     把关联并规整后的数据写入了“alldata”数据表


起始时间：6572.674710，从销售明细读取所有记录获取记录数：81.736487；关联客户、品牌和区域信息获得处理后的记录：97.349337；处理日期列：89.625873；写入内存数据库：587.361965；从存储数据库读取汇总销售数据：1.058238。


In [3]:
from imp4nb import *

cnx = lite.connect('data\\quandan.db')

df = pd.read_sql_query("select * from xiaoshoumingxi",cnx)
print(len(df))

df = pd.read_sql_query("select * from alldata",cnx)
print(len(df))


cnx.close()

1444120


1434642
